## Day 4: Giant Squid
[link](https://adventofcode.com/2021/day/4)

### Parsing

In [ ]:
#!value --name sampleRaw

7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7

Let us not invent anything sophisticated for parsing data. Good old `String.Split` should be fine.

When parsing, we construct a jagged array (array of arrays) which then can simply be converted to a two-dimensional array. F# slices feature should do the trick today.

Additionally, in type `Board` we will store the info about marked values.

In [ ]:
#load "../common.fsx"

In [ ]:
type Draws = int list
type Number = | Number of int * bool
type Board = | Board of Number[,]
let parse (input : string) = 
    let split = input.Split([|"\r\n\r\n"; "\n\n"|], StringSplitOptions.RemoveEmptyEntries)
    let draws = split.[0].Split(",") |> Seq.map int |> List.ofSeq
    let boards = 
        split 
        |> Seq.skip 1 
        |> Seq.map (fun boardRaw -> 
                        boardRaw.Split([|'\n'; '\r'|], StringSplitOptions.RemoveEmptyEntries)
                        |> Array.map(fun boardRow -> boardRow.Split(' ', StringSplitOptions.RemoveEmptyEntries))
                        |> array2D
                        |> Array2D.map (fun number -> Number(int number, false))
                    )
        |> Seq.map Board
        |> Array.ofSeq
    boards, draws


For beauty and visibility, override the formatting rule for the `Board` type 

In [ ]:
#load "../formatting.fsx"
let private formatBoardElement el =
    match el with
    | Number (num, true) -> $"<u><b>{num}</b></u>"
    | Number (num, false) -> string num    

Formatter.Register<Board>((fun (Board (board)) -> formatTable formatBoardElement board), "text/html")

In [ ]:
#!share sampleRaw --from value
let (sampleBoards, sampleDraws) = parse sampleRaw
sampleBoards

index value 0 22 13 17 11 0 8 2 23 4 24 21 9 14 16 7 6 10 3 18 5 1 12 20 15 19 1 3 15 0 2 22 9 18 13 17 5 19 8 7 25 23 20 11 10 24 4 14 21 16 12 6 2 14 21 17 24 4 10 16 15 9 19 18 8 23 26 20 22 11 13 6 5 2 0 12 3 7

### Helpers

Each stage of the game will be represented by the following type. During the game numbers will be drown from `Draws`, all the `Boards` marked and in case of some Board winning  this Board will be removed from `Boards` and added to `Winners` along with the number.

In [ ]:
type State = { Boards : Board list; Winners : (Board * int) list; Draws : Draws }

A separate function to mark a number in a Board.

In [ ]:
let mark number (Board board) = 
    Array2D.map (fun (Number (num, marked)) -> Number (num, marked || (num = number))) board
    |> Board

A separate function to check if a Board has at least one complete row or column of marked numbers

In [ ]:
let check (Board board) = 
    let check vector = 
        Seq.forall (fun (Number (_, marked)) -> marked) vector
    [|0..(Array2D.length1 board - 1)|]
    |> Array.exists (fun i -> board.[i, *] |> check)
    ||
    [|0..(Array2D.length2 board - 1)|]
    |> Array.exists (fun j -> board.[*, j] |> check)

For example:

In [ ]:
let notMarkedYet = 
    [24; 23; 2; 8; 16]
    |> List.fold (fun acc num -> mark num acc) sampleBoards.[0]
    |> displayPipe

notMarkedYet
    |> check

22,13,17,11,0
8,2,23,4,24
21,9,14,16,7
6,10,3,18,5
1,12,20,15,19


False

In [ ]:
notMarkedYet |> mark 4
|> displayPipe
|> check
|> display

22,13,17,11,0
8,2,23,4,24
21,9,14,16,7
6,10,3,18,5
1,12,20,15,19


True

In [ ]:
[0; 7; 5; 19]
|> List.fold (fun acc num -> mark num acc) notMarkedYet
|> displayPipe
|> check
|> display


22,13,17,11,0
8,2,23,4,24
21,9,14,16,7
6,10,3,18,5
1,12,20,15,19


True

### Game

The game stops when there is nothing more to draw or no boards was left. Otherwise, mark (immutably) numbers on the boards, then move winners to another list, and repeat. When game stops, the `Winner` list will consist of all the winners in reverse order. The immutable nature of all processes guarantees that they will have the state they had when won.

In [ ]:
let mutable demo = true

let rec game (state : State) = 
    match state with
    | { Draws = [] } -> state
    | { Boards = [] } -> state
    | { Draws = draw::drawsRest; Boards = boards} ->
        if (demo) then $"Draw {draw}." |> display |> ignore
        let markedBoards = List.map (mark draw) boards
        let (wonBoards, notWonBoards) = List.partition check markedBoards
        if (demo && not (List.isEmpty wonBoards)) then  
            $"Winners:" |> display |> ignore
            wonBoards |> display |> ignore
        let wonBoardsWithCurrentDraw = wonBoards |> List.map (fun b -> b, draw)
        game { Boards = notWonBoards; Winners = wonBoardsWithCurrentDraw @ state.Winners; Draws = drawsRest }

In [ ]:
demo <- true
game {Draws = sampleDraws; Boards = sampleBoards |> List.ofArray; Winners = []} |> ignore

Draw 7.

Draw 4.

Draw 9.

Draw 5.

Draw 11.

...

Winners:

index value 0 22 13 17 11 0 8 2 23 4 24 21 9 14 16 7 6 10 3 18 5 1 12 20 15 19

Draw 13.

Winners:

index value 0 3 15 0 2 22 9 18 13 17 5 19 8 7 25 23 20 11 10 24 4 14 21 16 12 6

### Actual data

Now, we can simulate the game for the actual input once. The first and the last winning Boards will be found as the last and the first elements of `Winners` respectively.

In [ ]:
#!value --name inputRaw --from-file ./data

In [ ]:
demo <- false

#!share inputRaw --from value
let (boards, draws) = parse inputRaw
let {State.Winners = winners} = game {Draws = draws; Boards = boards |> List.ofArray; Winners = []}

First winning Board:

In [ ]:
let (private firstWinningBoard, private firstWinningDraw) = List.last winners
firstWinningBoard |> display
firstWinningDraw |> display

let calculateScore (Board winningBoard) winningDraw = 
    winningBoard |> Array2D.toArray 
    |> Array.choose (fun (Number (num, marked)) -> if (marked) then None else Some num) 
    |> Array.sum
    |> (*) winningDraw

calculateScore firstWinningBoard firstWinningDraw


24,9,94,69,65
97,84,85,53,5
92,11,61,77,8
21,75,33,57,63
43,68,55,52,93


75

65325

Last winning Board:

In [ ]:
let (private lastWinningBoard, private lastWinningDraw) = List.head winners
lastWinningBoard |> display
lastWinningDraw |> display

calculateScore lastWinningBoard lastWinningDraw


84,86,77,97,28
37,87,2,93,5
16,64,35,61,27
8,3,36,10,73
31,65,94,63,13


16

4624